In [41]:
import ee
import geemap
import math
import pandas as pd
import matplotlib.pyplot as plt
import os 
from PIL import Image, ImageDraw, ImageFont
import matplotlib.colors as mcolors
import numpy as np

In [96]:
def HarmonicFuncLandsat(file,anio,omega,intervalo_dias):
    
    start_date = '{0}-04-01'.format(anio)
    end_date = '{0}-11-30'.format(anio)
    region = geemap.shp_to_ee(str('../data/Polygons/' + file))
    geometry = region.geometry()

    #cdl = ee.Image('USDA/NASS/CDL/{0}'.format(anio)).select('cropland')
    #masked = cdl.updateMask(cdl.eq(5).Or(cdl.eq(26)).Or(cdl.eq(254)))
    #remasked = masked.where(masked.eq(5), 0).where(masked.eq(26), 1).where(masked.eq(254), 1)

    # Import Landsat imagery.
    landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

    # Functions to rename Landsat 7 and 8 images.
    def renameL7(img):
        return img.rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1',
            'SWIR2', 'TEMP1', 'ATMOS_OPACITY', 'QA_CLOUD',
            'ATRAN', 'CDIST',
            'DRAD', 'EMIS', 'EMSD', 'QA', 'TRAD', 'URAD',
            'QA_PIXEL',
            'QA_RADSAT'
        ])

    def renameL8(img):
        return img.rename(['AEROS', 'BLUE', 'GREEN', 'RED', 'NIR',
            'SWIR1',
            'SWIR2', 'TEMP1', 'QA_AEROSOL', 'ATRAN', 'CDIST',
            'DRAD', 'EMIS',
            'EMSD', 'QA', 'TRAD', 'URAD', 'QA_PIXEL', 'QA_RADSAT'
        ])

    # Functions to mask out clouds, shadows, and other unwanted features.
    def addMask(img):
        # Bit 0: Fill
        # Bit 1: Dilated Cloud
        # Bit 2: Cirrus (high confidence) (L8) or unused (L7)
        # Bit 3: Cloud
        # Bit 4: Cloud Shadow
        # Bit 5: Snow
        # Bit 6: Clear
        #        0: Cloud or Dilated Cloud bits are set
        #        1: Cloud and Dilated Cloud bits are not set
        # Bit 7: Water
        clear = img.select('QA_PIXEL').bitwiseAnd(64).neq(0)
        clear = clear.updateMask(clear).rename(['pxqa_clear'])

        water = img.select('QA_PIXEL').bitwiseAnd(128).neq(0)
        water = water.updateMask(water).rename(['pxqa_water'])

        cloud_shadow = img.select('QA_PIXEL').bitwiseAnd(16).neq(0)
        cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename([
            'pxqa_cloudshadow'
        ])

        snow = img.select('QA_PIXEL').bitwiseAnd(32).neq(0)
        snow = snow.updateMask(snow).rename(['pxqa_snow'])

        masks = ee.Image.cat([
            clear, water, cloud_shadow, snow
        ])

        return img.addBands(masks)

    def maskQAClear(img):
        return img.updateMask(img.select('pxqa_clear'))

    # Function to add GCVI as a band.
    def addVIs(img):
        gcvi = img.expression('(nir / green) - 1', {
          'nir': img.select('NIR'),
          'green': img.select('GREEN')
        }).select([0], ['GCVI'])


        return ee.Image.cat([img, gcvi])

    # Función para aplicar los factores de escala a las bandas
    def apply_scale_factors_L8(image):
        optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        return image.addBands(optical_bands, overwrite=True)

    def apply_scale_factors_L7(image):
        optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        return image.addBands(optical_bands, overwrite=True)

    # Pull Landsat 7 and 8 imagery over the study area between start and end dates.
    landsat7coll = landsat7 \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .map(apply_scale_factors_L7) \
        .map(renameL7)

    landsat8coll = landsat8 \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry) \
        .map(apply_scale_factors_L8) \
        .map(renameL8)

    # Merge Landsat 7 and 8 collections.
    landsat = landsat7coll.merge(landsat8coll) \
        .sort('system:time_start')


    # Mask out non-clear pixels, add VIs and time variables.
    landsat = landsat.map(addMask) \
        .map(maskQAClear) \
        .map(addVIs)

    #landsat.map(lambda image: image.updateMask(remasked.eq(0).Or(remasked.eq(1))))

    # Function that adds time band to an image.
    def addTimeUnit(image, refdate):
        date = image.date()

        dyear = date.difference(refdate, 'year')
        t = image.select(0).multiply(0).add(dyear).select([0], ['t']) \
            .float()

        imageplus = image.addBands(t)

        return imageplus


    # Function that adds harmonic basis to an image.
    def addHarmonics(image, omega, refdate):
        image = addTimeUnit(image, refdate)
        timeRadians = image.select('t').multiply(2 * math.pi * omega)
        timeRadians2 = image.select('t').multiply(4 * math.pi *
        omega)

        return image \
            .addBands(timeRadians.cos().rename('cos')) \
            .addBands(timeRadians.sin().rename('sin')) \
            .addBands(timeRadians2.cos().rename('cos2')) \
            .addBands(timeRadians2.sin().rename('sin2')) \
            .addBands(timeRadians.divide(timeRadians) \
                .rename('constant'))

    def apply_harmonics(image):
        return addHarmonics(image, omega, start_date)

    landsatPlus = landsat.map(apply_harmonics)

    def arrayimgHarmonicRegr(harmonicColl, dependent, independents):
        independents = ee.List(independents)
        dependent = ee.String(dependent)

        regression = harmonicColl \
            .select(independents.add(dependent)) \
            .reduce(ee.Reducer.robustLinearRegression(independents.length(), 1))

        return regression

    def imageHarmonicRegr(harmonicColl, dependent, independents):
        hregr = arrayimgHarmonicRegr(harmonicColl, dependent, independents)

        independents = ee.List(independents)
        dependent = ee.String(dependent)

        def func_mxk(b):
            return dependent.cat(ee.String('_')).cat(ee.String(b))

        newNames = independents.map(func_mxk)

        imgCoeffs = hregr.select('coefficients') \
            .arrayProject([0]) \
            .arrayFlatten([independents]) \
            .select(independents, newNames)

        return imgCoeffs

    def apply_imageHarmonicRegr(band):
        return imageHarmonicRegr(harmonicColl, band, independents)

    def getHarmonicCoeffs(harmonicColl, bands, independents):
        coefficients = ee.ImageCollection.fromImages(
            bands.map(apply_imageHarmonicRegr)
        )
        return coefficients.toBands()

    # Definir harmonicColl antes de usarla, si no está definida antes en tu código
    harmonicColl = landsatPlus
    # Aplicar la función corregida
    bands = ee.List(['NIR', 'SWIR1', 'SWIR2', 'GCVI'])  # Cambié la lista a un objeto ee.List
    independents = ee.List(['constant', 'cos', 'sin', 'cos2', 'sin2'])
    harmonics = getHarmonicCoeffs(harmonicColl, bands, independents)

    harmonics = harmonics.clip(geometry)
    harmonics = harmonics.multiply(10000).toInt32()

    # Compute fitted values.
    gcviHarmonicCoefficients = harmonics \
        .select([
            '3_GCVI_constant', '3_GCVI_cos',
            '3_GCVI_sin', '3_GCVI_cos2', '3_GCVI_sin2'
        ]) \
        .divide(10000)

    def add_fitted_band(image):
        return image.addBands(
            image.select(independents) \
            .multiply(gcviHarmonicCoefficients) \
            .reduce('sum') \
            .rename('fitted')
        )

    fittedHarmonic = landsatPlus.map(add_fitted_band) 


    date_range = ee.DateRange(start_date, end_date)

    # Definir la función para calcular el valor ajustado de GCVI en un día específico
    def harmonic_fit(date, omega, harmonic_coefficients, independents):
        date = ee.Date(date)
        time = date.difference(ee.Date(start_date), 'year')

        #omega = 1.5
        timeRadians = ee.Image.constant(time).multiply(2 * math.pi * omega)
        timeRadians2 = ee.Image.constant(time).multiply(4 * math.pi * omega)

        # Crear las bandas de senos y cosenos
        cos = timeRadians.cos().rename('cos')
        sin = timeRadians.sin().rename('sin')
        cos2 = timeRadians2.cos().rename('cos2')
        sin2 = timeRadians2.sin().rename('sin2')
        constant = ee.Image.constant(1).rename('constant')

        # Combinar las bandas
        harmonics = ee.Image.cat([constant, cos, sin, cos2, sin2])

        # Calcular el valor ajustado usando los coeficientes
        fitted = harmonics.multiply(harmonic_coefficients).reduce('sum').rename('fitted_gcvi')

        return fitted.set('system:time_start', date.millis())

    # Crear una lista de fechas diarias
    dates = ee.List.sequence(ee.Date(start_date).millis(), ee.Date(end_date).millis(), intervalo_dias * 24 * 60 * 60 * 1000)

    # Mapear la función sobre la lista de fechas para generar una colección de imágenes diarias
    daily_harmonics = ee.ImageCollection.fromImages(
        dates.map(lambda millis: harmonic_fit(ee.Date(millis),omega, gcviHarmonicCoefficients, independents))
    ) 

    def add_residuals(img):
         return img.addBands(img.select('GCVI').subtract(img.select('fitted')) \
                             .rename('residuals')) 

    residuals = fittedHarmonic.map(add_residuals) #.map(lambda image: image.updateMask(remasked.eq(0).Or(remasked.eq(1))))

    dependent = 'GCVI'
    harmonicIndependents = ['constant', 'cos', 'sin', 'cos2', 'sin2']
    n = residuals.select(dependent).count()
    dof = n.subtract(len(harmonicIndependents))

    # Obtener los residuos cuadráticos medios en un píxel.

    rmsr = arrayimgHarmonicRegr(harmonicColl, dependent, independents).select('residuals') \
        .arrayProject([0]) \
        .arrayFlatten([['rmsr']]) #.updateMask(remasked.eq(0).Or(remasked.eq(1)))

    # Suma de cuadrados de los residuos.
    rss = rmsr.pow(2).multiply(n)

    rmse = rmsr.pow(2).reduce(ee.Reducer.mean()).sqrt().rename(dependent + '_rmse').clip(region)

    # Varianza estimada = RSS/(n-p).
    sSquared = rss.divide(dof)

    # Varianza muestral de la variable dependiente.
    yVariance = fittedHarmonic.select(dependent).reduce(ee.Reducer.sampleVariance())

    # Coeficiente de regresión múltiple ajustado y cuadrado.
    r2 = ee.Image(1).subtract(sSquared.divide(yVariance)).rename(dependent + '_R2').clip(region)

    metrics = ee.ImageCollection.fromImages([r2, rmse])

    daily_harmonics = daily_harmonics.merge(metrics).toBands()

    geemap.ee_export_image_to_drive(daily_harmonics, 
                                    description='Harmonized_Landsat_{0}_{1}'.format(anio,file.replace('.shp','')), 
                                    folder='coeff_harmonicos_emma', 
                                    maxPixels=  1e11,
                                    region=region.geometry(), 
                                    scale=30)

In [144]:
anio = 2023

intervalo_dias = 3

crop = 'soybean'

omega = 2

#file = 'Dunteman_pol.shp'
file = 'Brautigam_pol.shp'
HarmonicFuncLandsat(file,anio,omega,intervalo_dias) 

In [127]:
HarmonicFuncLandsat(file,anio,omega,intervalo_dias) 